In [1]:
# %load_ext autoreload
# %autoreload 2
import sys
dirr = "../../src"
if dirr not in sys.path:
    sys.path.append(dirr)
%xmode Plain
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils as helper

Exception reporting mode: Plain


In [17]:
data = pd.read_csv("../../data/tennis/tennis.csv", header=None)
data.head(3)

,0,1,2,3,4
0,outlook,temp,humidity,windy,play
1,sunny,hot,high,false,no
2,sunny,hot,high,true,no


---
## Question 1
**a) and b)** Using Parallelism internally by the use of ``Pool`` in ``multiprocessing`` library. Look at ``rf.py`` for the exact details.

In [18]:
from rf import RandomForestClassifier

In [25]:
rf = RandomForestClassifier(max_depth=1, frac=0.7, num_of_trees=3, jobs=4)
rf.train(data)

In [28]:
data.head(7)

,0,1,2,3,4
0,outlook,temp,humidity,windy,play
1,sunny,hot,high,false,no
2,sunny,hot,high,true,no
3,overcast,hot,high,false,yes
4,rainy,mild,high,false,yes
5,rainy,cool,normal,false,yes
6,rainy,cool,normal,true,no


In [26]:
rf.predict(data.iloc[:7, :])

,0,1,2,3,4,5,6
0,yes,yes,yes,yes,yes,yes,yes


In [27]:
rf.preds

,0,0,0
0,yes,yes,yes
1,yes,yes,yes
2,yes,yes,yes
3,yes,yes,yes
4,yes,yes,yes
5,yes,yes,yes
6,yes,yes,yes


In [ ]:
for dt_roots in rf.roots:
    print (helper.parser(d))

---
## Question 1
**c)** Below are the plot representing the increase in performance with increase in jobs.

---
## Question 1
**d)** Using RF on IRIS data set.

---
## Question 1
**e)** Using 5 fold cross-validation on IRIS dataset.

---
## Question 4
Generating a dataset with random noise as Gaussian Distribution.

In [ ]:
def dataset_generator(m, c, samples = 50, sigma=10): 
    """assumes random noise comes from
    Gaussian Distribution
    """
    X = np.arange(samples)
    random_noise = np.random.normal(scale=sigma, size=samples)
    Y = m*X + c + random_noise
    return X, Y

In [ ]:
X, Y = dataset_generator(m=2, c=43, samples=50, sigma=10)
plt.scatter(X, Y)

---
## Question 5
Look at the code in ``utils.py`` for the algorithm used. Basically I used Linear congruential generator.

In [ ]:
import utils as helper
y = helper.random_list(N = 100, samples = 10000)
plt.hist(y)